In [44]:
import os
import cv2
import torch
import numpy as np

In [49]:
model = torch.hub.load('WongKinYiu/yolov7', 'custom', 'yolov7.pt',source='github', force_reload=False)

Using cache found in C:\Users\user/.cache\torch\hub\WongKinYiu_yolov7_main

                 from  n    params  module                                  arguments                     
  0                -1  1       928  models.common.Conv                      [3, 32, 3, 1]                 
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  5                -2  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  6                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  7                -1  1     36992  models.common.Conv              

Adding autoShape... 


In [50]:
def frame_stack(frames,rows,cols):
    h,w,c = frames[0].shape             #cameras count
    blank = np.zeros((h,w,c),np.uint8)  #balcnk image
    while len(frames) < rows*cols:       #if less than no.of cameras append new blank image loop
        frames.append(blank)

    grid_row =[]
    for r in range(rows):
        rows_frames = frames[r*cols:(r+1)*cols] 
        rows = np.hstack(rows_frames)
        grid_row.append(rows)

    grid = np.vstack(grid_row)
    return grid

In [52]:
#Hyoerparamters for video saving
out_dir = "output_images"
os.makedirs(out_dir,exist_ok=True)

w = 320
h = 240
grid_rows=2
grid_cols=2

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out_path = os.path.join(out_dir,'multiple_webcame.avi')

out = cv2.VideoWriter(out_path,fourcc,20.0,(w*grid_cols,h*grid_rows))

In [53]:
#list of cameras
camera_sources = [0,0,'yoga.mp4']

#loop for video capture
caps = [cv2.VideoCapture(src) for src in camera_sources]

In [54]:


while True:
    all_frames=[]
    for cap in caps:
        ret, frame = cap.read()
        if ret == True:   
            #RGB
            frame = cv2.resize(frame, (w,h))
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
            #model
            result = model(rgb_frame)
    
            #writable
            result.imgs[0] = result.imgs[0].copy()
    
            #render
            rendered_result = result.render()[0]
    
            #BGR
            bgr_frame = cv2.cvtColor(rendered_result,cv2.COLOR_RGB2BGR)

            all_frames.append(bgr_frame)
      
        else:
            #display blank image if disconnected or camera not found!
            frames = np.zeros((h,w,3),np.uint8)
            print("Camera Not Found!")
            break


    grid_frame = frame_stack(all_frames,grid_rows, grid_cols)
    cv2.imshow('Multiple webcame object detection',grid_frame)
    out.write(grid_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break  

            
for cap in caps:
    cap.release()
out.release()
cv2.destroyAllWindows()